In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-26 23:38:42.901083: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 23:38:42.927993: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 23:38:42.928018: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 23:38:42.928044: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 23:38:42.933761: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-26 23:38:44.607280: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 23:38:44.633450: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 23:38:44.636387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [5]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (299, 299)


In [6]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [7]:
IMG_HEIGHT, IMG_WIDTH = 299,299

In [8]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-26 23:38:44.729948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 23:38:44.732385: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 23:38:44.734599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [9]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [10]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.xception.preprocess_input

In [11]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [12]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 299,299
base_model = apps.Xception(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [12]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: block1_conv1, False
Layer: block1_conv1_bn, False
Layer: block1_conv1_act, False
Layer: block1_conv2, False
Layer: block1_conv2_bn, False
Layer: block1_conv2_act, False
Layer: block2_sepconv1, False
Layer: block2_sepconv1_bn, False
Layer: block2_sepconv2_act, False
Layer: block2_sepconv2, False
Layer: block2_sepconv2_bn, False
Layer: conv2d, False
Layer: block2_pool, False
Layer: batch_normalization, False
Layer: add, False
Layer: block3_sepconv1_act, False
Layer: block3_sepconv1, False
Layer: block3_sepconv1_bn, False
Layer: block3_sepconv2_act, False
Layer: block3_sepconv2, False
Layer: block3_sepconv2_bn, False
Layer: conv2d_1, False
Layer: block3_pool, False
Layer: batch_normalization_1, False
Layer: add_1, False
Layer: block4_sepconv1_act, False
Layer: block4_sepconv1, False
Layer: block4_sepconv1_bn, False
Layer: block4_sepconv2_act, False
Layer: block4_sepconv2, False
Layer: block4_sepconv2_bn, False
Layer: conv2d_2, False
Layer: block4_pool, False
L

In [13]:
len(base_model.layers)

132

In [14]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [16]:
aug_layer = utils.return_data_aug_layer_for_eff_net(random_width=0.3, random_height=0.3, random_zoom=0.3, random_rotation=0.3)

In [17]:
from keras.src.regularizers import l2
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name='Input_layer')
x = aug_layer(inputs)
x = base_model(x, training=False)
x = GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)


x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

outputs = Dense(No_of_classes, activation='softmax', dtype=tf.float32)(x)

model_1 = Model(inputs, outputs, name='Xception_new')

model_1.summary()

Model: "Xception_new"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization_4 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                      

In [18]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [19]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=3) 

In [20]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [21]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 10, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/10


2024-07-26 22:45:19.955248: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-26 22:45:20.012879: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-26 22:45:21.586990: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x76e7dc015d50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-26 22:45:21.587007: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-26 22:45:21.601127: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-26 22:45:21.683781: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 13.0321 - accuracy: 0.5646
Epoch 1: val_loss improved from inf to 10.22709, saving model to Trained_Models/Xception_new
74/74 [==============================] - 76s 975ms/step - loss: 13.0321 - accuracy: 0.5646 - val_loss: 10.2271 - val_accuracy: 0.8239 - lr: 0.0010
Epoch 2/10
74/74 [==============================] - ETA: 0s - loss: 7.7175 - accuracy: 0.7048
Epoch 2: val_loss improved from 10.22709 to 5.83202, saving model to Trained_Models/Xception_new
74/74 [==============================] - 70s 938ms/step - loss: 7.7175 - accuracy: 0.7048 - val_loss: 5.8320 - val_accuracy: 0.9318 - lr: 0.0010
Epoch 3/10
74/74 [==============================] - ETA: 0s - loss: 4.5593 - accuracy: 0.7154
Epoch 3: val_loss improved from 5.83202 to 3.58261, saving model to Trained_Models/Xception_new
74/74 [==============================] - 63s 844ms/step - loss: 4.5593 - accuracy: 0.7154 - val_loss: 3.5826 - val_accuracy: 0.9644 - lr: 0.0010
Epoch

In [22]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 88ms/step - loss: 1.0995 - accuracy: 0.7314


[1.099496603012085, 0.7313609719276428]

In [23]:
start_epoch = 10
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10,initial_epoch=start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 11/20
74/74 [==============================] - ETA: 0s - loss: 1.1123 - accuracy: 0.7421
Epoch 11: val_loss improved from 0.93299 to 0.83094, saving model to Trained_Models/Xception_new
74/74 [==============================] - 57s 759ms/step - loss: 1.1123 - accuracy: 0.7421 - val_loss: 0.8309 - val_accuracy: 0.9209 - lr: 0.0010
Epoch 12/20
74/74 [==============================] - ETA: 0s - loss: 1.0869 - accuracy: 0.7306
Epoch 12: val_loss did not improve from 0.83094
74/74 [==============================] - 51s 686ms/step - loss: 1.0869 - accuracy: 0.7306 - val_loss: 1.3616 - val_accuracy: 0.4263 - lr: 0.0010
Epoch 13/20
74/74 [==============================] - ETA: 0s - loss: 1.0763 - accuracy: 0.7171
Epoch 13: val_loss did not improve from 0.83094
74/74 [==============================] - 50s 671ms/step - loss: 1.0763 - accuracy: 0.7171 - val_loss: 1.2301 - val_accuracy: 0.6231 - lr: 0.0010
Epoch 14/20
74/74 [==============================] - ETA: 0s - loss: 1.0577 - accuracy:

In [24]:
base_model.trainable = True

In [25]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [26]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [27]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 62ms/step - loss: 0.8454 - accuracy: 0.7633


[0.845353901386261, 0.7633135914802551]

In [28]:
start_epoch = 20
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Xception_new"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization_4 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                      

In [29]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/30
74/74 [==============================] - ETA: 0s - loss: 0.8524 - accuracy: 0.7539
Epoch 21: val_loss did not improve from 0.67331
74/74 [==============================] - 54s 650ms/step - loss: 0.8524 - accuracy: 0.7539 - val_loss: 5.4527 - val_accuracy: 0.0959 - lr: 3.0000e-04
Epoch 22/30
74/74 [==============================] - ETA: 0s - loss: 0.7090 - accuracy: 0.7916
Epoch 22: val_loss did not improve from 0.67331
74/74 [==============================] - 48s 652ms/step - loss: 0.7090 - accuracy: 0.7916 - val_loss: 1.7467 - val_accuracy: 0.2394 - lr: 3.0000e-04
Epoch 23/30
74/74 [==============================] - ETA: 0s - loss: 0.6371 - accuracy: 0.8141
Epoch 23: val_loss improved from 0.67331 to 0.59114, saving model to Trained_Models/Xception_new
74/74 [==============================] - 57s 770ms/step - loss: 0.6371 - accuracy: 0.8141 - val_loss: 0.5911 - val_accuracy: 0.8744 - lr: 3.0000e-04
Epoch 24/30
74/74 [==============================] - ETA: 0s - loss: 0.5950

In [31]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 66ms/step - loss: 0.5313 - accuracy: 0.8379


[0.531337559223175, 0.8378698229789734]

In [32]:
start_epoch = 30
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [33]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0001), metrics = ['accuracy'])
model_1.summary()

Model: "Xception_new"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization_4 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                      

In [34]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 31/40
74/74 [==============================] - ETA: 0s - loss: 0.4220 - accuracy: 0.8789
Epoch 31: val_loss did not improve from 0.23682
74/74 [==============================] - 45s 511ms/step - loss: 0.4220 - accuracy: 0.8789 - val_loss: 0.4918 - val_accuracy: 0.8734 - lr: 1.0000e-04
Epoch 32/40
74/74 [==============================] - ETA: 0s - loss: 0.3774 - accuracy: 0.8933
Epoch 32: val_loss improved from 0.23682 to 0.23257, saving model to Trained_Models/Xception_new
74/74 [==============================] - 53s 709ms/step - loss: 0.3774 - accuracy: 0.8933 - val_loss: 0.2326 - val_accuracy: 0.9683 - lr: 1.0000e-04
Epoch 33/40
74/74 [==============================] - ETA: 0s - loss: 0.3419 - accuracy: 0.8996
Epoch 33: val_loss did not improve from 0.23257
74/74 [==============================] - 40s 545ms/step - loss: 0.3419 - accuracy: 0.8996 - val_loss: 0.2846 - val_accuracy: 0.9347 - lr: 1.0000e-04
Epoch 34/40
74/74 [==============================] - ETA: 0s - loss: 0.3478

In [36]:
start_epoch = 40
model_1.evaluate(test_datagen)
for layer in base_model.layers[-100:]:
    layer.trainable = True

27/27 [==============================] - 2s 64ms/step - loss: 0.3779 - accuracy: 0.9041


In [37]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0000075), metrics = ['accuracy'])
model_1.summary()

Model: "Xception_new"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization_4 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                      

In [38]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+15, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 41/55
74/74 [==============================] - ETA: 0s - loss: 0.2441 - accuracy: 0.9437
Epoch 41: val_loss did not improve from 0.15178
74/74 [==============================] - 52s 575ms/step - loss: 0.2441 - accuracy: 0.9437 - val_loss: 0.1676 - val_accuracy: 0.9703 - lr: 7.5000e-06
Epoch 42/55
74/74 [==============================] - ETA: 0s - loss: 0.2404 - accuracy: 0.9382
Epoch 42: val_loss did not improve from 0.15178
74/74 [==============================] - 42s 556ms/step - loss: 0.2404 - accuracy: 0.9382 - val_loss: 0.1715 - val_accuracy: 0.9654 - lr: 7.5000e-06
Epoch 43/55
74/74 [==============================] - ETA: 0s - loss: 0.2180 - accuracy: 0.9496
Epoch 43: val_loss did not improve from 0.15178
74/74 [==============================] - 44s 596ms/step - loss: 0.2180 - accuracy: 0.9496 - val_loss: 0.1615 - val_accuracy: 0.9624 - lr: 7.5000e-06
Epoch 44/55
74/74 [==============================] - ETA: 0s - loss: 0.2194 - accuracy: 0.9513
Epoch 44: val_loss did not im

In [39]:
start_epoch = 55
for layer in base_model.layers[-125:]:
    layer.trainable = True

In [40]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0000005), metrics = ['accuracy'])
model_1.summary()

Model: "Xception_new"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization_4 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                      

In [41]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 56/65
45/74 [=================>............] - ETA: 46s - loss: 0.1994 - accuracy: 0.9569

2024-07-26 23:37:23.089954: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-07-26 23:37:23.090006: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-07-26 23:37:23.212924: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-07-26 23:37:23.212970: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU

52/74 [====================>.........] - ETA: 34s - loss: 0.2032 - accuracy: 0.9549

2024-07-26 23:37:41.568625: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 23.50MiB (rounded to 24647168)requested by op Xception_new/xception/block13_sepconv2/separable_conv2d/depthwise
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-07-26 23:37:41.568826: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-07-26 23:37:41.568850: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 401, Chunks in use: 401. 100.2KiB allocated for chunks. 100.2KiB in use in bin. 12.6KiB client-requested in use in bin.
2024-07-26 23:37:41.568865: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 61, Chunks in use: 61. 30.8KiB allocated for chunks. 30.8KiB in use in bin. 30.5KiB client-requested in use 

ResourceExhaustedError: Graph execution error:

Detected at node Xception_new/xception/block13_sepconv2/separable_conv2d/depthwise defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", lin

In [13]:
loaded = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Xception_new_90.41")
loaded.evaluate(test_datagen)

2024-07-26 23:39:20.694685: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-26 23:39:20.797285: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


27/27 [==============================] - 4s 79ms/step - loss: 0.3779 - accuracy: 0.9041


[0.3778638541698456, 0.9041420221328735]